# 2.1 - Prompts and Output Parsers 

## Setup

### Install dependencies

In [1]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pydantic~=2.9 pypdf~=5.1 --upgrade --quiet
%pip install langchain~=0.3.7 langchain_openai~=0.2.6 langchain_community~=0.3.5 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Load environment variables

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")

### Setup Models

In [3]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
api_version = "2024-10-01-preview"
llm = AzureChatOpenAI(deployment_name="gpt-4o", temperature=0.0, openai_api_version=api_version)
embedding_model = AzureOpenAIEmbeddings(model="text-embedding-3-large", openai_api_version=api_version)


## Prompts

### Prompt template - for any type of LM (instruct or base) and simple use cases

In [4]:
from langchain.prompts import PromptTemplate

template_string = ("Translate the text that is delimited by triple backticks into a style that "
                   "is {style}. text: ```{text}```")

prompt = PromptTemplate.from_template(template_string)
prompt.format(style="geeky", text="Hello, world!")

'Translate the text that is delimited by triple backticks into a style that is geeky. text: ```Hello, world!```'

### Chat prompt template - for chat-based LLMs
Basically a chat prompt template is a list of messages, where each message in itself can be a template.

In [5]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = "Translate user input into a style that is {style}."

prompt_template_typed = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{input}"),
])

# Use can also use this shorthand to create the same prompt template    
prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("human", "{input}"),
])


In [6]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style'], input_types={}, partial_variables={}, template='Translate user input into a style that is {style}.')

In [7]:
prompt_template.messages[0].prompt.input_variables

['style']

In [11]:
customer_style = "German in a calm and respectful tone"

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_messages(
    style=customer_style,
    input=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))
print(type(customer_messages[1]))

<class 'list'>
<class 'langchain_core.messages.system.SystemMessage'>
<class 'langchain_core.messages.human.HumanMessage'>


In [12]:
print(customer_messages[0])

content='Translate user input into a style that is German in a calm and respectful tone.' additional_kwargs={} response_metadata={}


In [13]:
# Call the LLM to translate to the style of the customer message
customer_response = llm.invoke(customer_messages)
print(customer_response.content)

Ich bin verärgert darüber, dass der Deckel meines Mixers sich gelöst hat und mein Küchenwände mit Smoothie bespritzt hat. Zu allem Überfluss deckt die Garantie nicht die Kosten für die Reinigung meiner Küche ab. Ich benötige jetzt Ihre Hilfe, mein Freund.


#### Try another example

In [14]:
customer_style = "The same style as the user"

customer_messages = prompt_template.format_messages(
    style=customer_style,
    input=customer_email)

customer_response = llm.invoke(customer_messages)
print(customer_response.content)

Arrr, that be a right mess, matey! Sounds like ye got caught in a storm of smoothie chaos. First, grab a mop and some rags to swab the decks, er, walls. A bit of warm water and soap should help tame the fruity seas. As for that warranty, it be a scallywag's trick, but maybe a call to the company might get ye some treasure in the form of a replacement lid. Keep yer chin up, and may the winds be ever in yer favor!


### MessagesPlaceholder

What if we wanted the user to pass in a list of messages that we would slot into a particular spot? This is how you use MessagesPlaceholder. A typical use case for this in when you need to include **_conversation history_** in the prompt.

In [15]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("history")
])

prompt_template.invoke({"history": [HumanMessage(content="hi!"), AIMessage(content="Sup fam?")]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}), AIMessage(content='Sup fam?', additional_kwargs={}, response_metadata={})])

In [16]:
# Alternative, shorthand syntax
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("placeholder", "{history}") # <-- This is the changed part
])

<br/>

----
<br/>


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

## The most common parser - String output parsing

In [17]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()
llm.invoke("")

parsed_response = str_parser.invoke(customer_response)
print(parsed_response)

Arrr, that be a right mess, matey! Sounds like ye got caught in a storm of smoothie chaos. First, grab a mop and some rags to swab the decks, er, walls. A bit of warm water and soap should help tame the fruity seas. As for that warranty, it be a scallywag's trick, but maybe a call to the company might get ye some treasure in the form of a replacement lid. Keep yer chin up, and may the winds be ever in yer favor!


### Structured output parsing

In [18]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [19]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [20]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [21]:
messages = prompt_template.format_messages(text=customer_review)
response = llm.invoke(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```


In [23]:
type(response.content)

str

In [24]:
# You will get an error by running this line of code 
# because content is not a dictionary, but a string
response.content.get('gift') # This will raise an error - comment out to proceed

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [25]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema]

In [26]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [27]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [28]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                  format_instructions=format_instructions)

In [29]:
response = llm.invoke(messages)
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [30]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [31]:
output_dict = output_parser.parse(response.content)

In [32]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [33]:
type(output_dict)

dict

In [ ]:
output_dict.get('delivery_days')